In [ ]:
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os
import matplotlib.pyplot as plt
from scipy.io import loadmat
import bisect
import numpy as np
torch.manual_seed(1)
err_all=[]
num_list=[10,20,50,100]
for i in range(len(num_list)):
    N=num_list(i)
    M=0.1*N*N
    print(N)
    class PINN(nn.Module):
        def __init__(self):
            super().__init__()
            self.model1 = nn.Sequential(
                nn.Linear(2, N),
                nn.Tanh(),
                nn.Linear(N, M),
                nn.Tanh(),
                nn.Linear(M, 2)
            )

        def forward(self, x):
            output = self.model1(x)
            return output


    class MSE0b(nn.Module):
        def __init__(self):
            super().__init__()

        def forward(self, x, y):
            output = torch.sum((x - y) ** 2)/len(x)
            return output


    class MSEf(nn.Module):
        def __init__(self):
            super().__init__()

        def forward(self, x):
            output = torch.sum(x ** 2)/len(x)
            return output


    # 模型初始化
    pinn = PINN().cuda()
    optimizer = torch.optim.LBFGS(pinn.parameters(), lr=0.2)
    init_loss = MSE0b().cuda()
    boundary_loss = MSE0b().cuda()
    internal_loss = MSEf().cuda()
    
        # 初值训练点
    N0 = 50
    init_data = (torch.rand(N0, 2) - 0.5) * 10
    init_data[:, 0:1] = torch.zeros((N0, 1))
    init_data = init_data.cuda()
    # init_data.requires_grad_()

    init_target = torch.zeros((N0, 2))
    init_target[:, 0:1] = 2 / torch.cosh(init_data[:, 1:2])
    init_target = init_target.cuda()

    # 边界训练点
    Nb = 50
    sample_time = torch.rand((Nb, 1))*torch.pi/2
    bd1_data = torch.full((Nb, 2), 5.)
    bd1_data[:, 0:1] = sample_time
    bd2_data = torch.full((Nb, 2), -5.)
    bd2_data[:, 0:1] = sample_time
    bd1_data = bd1_data.cuda()
    bd2_data = bd2_data.cuda()
    # bd1_data.requires_grad_()
    # bd2_data.requires_grad_()

    # 内部训练点
    Nf = 20000
    sample_x = (torch.rand(Nf, 1) - 0.5) * 10
    sample_time2 = torch.rand((Nf, 1))*torch.pi/2
    internal_data = torch.zeros((Nf, 2))
    internal_data[:, 0:1] = sample_time2
    internal_data[:, 1:2] = sample_x
    internal_data = internal_data.cuda()
    # internal_data.requires_grad_()

    # 求导梯度
    bd_u_grad_out = torch.Tensor([[1, 0]*Nb]).reshape((-1, 2)).cuda()
    bd_v_grad_out = torch.Tensor([[0, 1]*Nb]).reshape((-1, 2)).cuda()

    internal_u_grad_out = torch.Tensor([[1, 0]*Nf]).reshape((-1, 2)).cuda()
    internal_v_grad_out = torch.Tensor([[0, 1]*Nf]).reshape((-1, 2)).cuda()
    
    pinn.train()
    def closure():
        # 初值误差
        init_data_ = init_data.clone()
        init_data_.requires_grad_()
        init_y = pinn(init_data_)
        mse0 = init_loss(init_y, init_target)

        # 边值误差
        bd1_data_ = bd1_data.clone()
        bd1_data_.requires_grad_()
        bd2_data_ = bd2_data.clone()
        bd2_data_.requires_grad_()
        bd1_y = pinn(bd1_data_)
        bd2_y = pinn(bd2_data_)

        bd1_dudx = torch.autograd.grad(outputs=bd1_y, inputs=bd1_data_, grad_outputs=bd_u_grad_out, create_graph=True)[0][:, 1:2]
        bd1_dvdx = torch.autograd.grad(outputs=bd1_y, inputs=bd1_data_, grad_outputs=bd_v_grad_out, create_graph=True)[0][:, 1:2]

        bd2_dudx = torch.autograd.grad(outputs=bd2_y, inputs=bd2_data_, grad_outputs=bd_u_grad_out, create_graph=True)[0][:, 1:2]
        bd2_dvdx = torch.autograd.grad(outputs=bd2_y, inputs=bd2_data_, grad_outputs=bd_v_grad_out, create_graph=True)[0][:, 1:2]

        mseb = boundary_loss(bd1_dudx, bd2_dudx) + boundary_loss(bd1_dvdx, bd2_dvdx)

        # 内点误差
        internal_data_ = internal_data.clone()
        internal_data_.requires_grad_()
        internal_y = pinn(internal_data_)

        internal_du = torch.autograd.grad(outputs=internal_y, inputs=internal_data_, grad_outputs=internal_u_grad_out, create_graph=True)[0]
        internal_dv = torch.autograd.grad(outputs=internal_y, inputs=internal_data_, grad_outputs=internal_v_grad_out, create_graph=True)[0]

        internal_dudt = internal_du[:, 0:1]
        internal_dudx = internal_du[:, 1:2]
        internal_dvdt = internal_dv[:, 0:1]
        internal_dvdx = internal_dv[:, 1:2]

        internal_ddudxx = torch.autograd.grad(outputs=internal_dudx, inputs=internal_data_, grad_outputs=torch.ones_like(internal_dudx), create_graph=True)[0][:, 1:2]
        internal_ddvdxx = torch.autograd.grad(outputs=internal_dvdx, inputs=internal_data_, grad_outputs=torch.ones_like(internal_dudx), create_graph=True)[0][:, 1:2]

        uv_square = internal_y[:, 0:1] ** 2 + internal_y[:, 1:2] ** 2
        f_real = 0.5 * internal_ddudxx - internal_dvdt + uv_square * internal_y[:, 0:1]
        f_imag = 0.5 * internal_ddvdxx + internal_dudt + uv_square * internal_y[:, 1:2]

        msef = internal_loss(f_real) + internal_loss(f_imag)

        # 总共误差
        total_mse = mse0 + mseb + msef

        # 清空梯度+反向传播
        optimizer.zero_grad()
        total_mse.backward()

        return total_mse

    writer = SummaryWriter("PINN_log_2layers")
    #for epoch in range(2000):
    #    if epoch % 10 == 0 :
    #        print("Epoch: {}".format(epoch))
    #    writer.add_scalar("train_%d" % N, optimizer.step(closure).item(), epoch)

    #writer.close()
    
    exact_data = loadmat("NLS.mat")
    idx = bisect.bisect_right(exact_data['tt'][0], 1.00)
    t = exact_data['tt'][0][idx]
    x = exact_data['uu'][:, idx]
    x_axis = exact_data['x'][0]

    pinn.train(False)

    for epoch in range(2000):
        if epoch % 10 == 0 :
            pinn.train(False)
            print("Epoch: {}, begin evaluation.".format(epoch))
            with torch.no_grad():
                test_data = torch.full((len(x), 2), t)
                test_data[:, 1:2] = torch.tensor(exact_data['x'].T)
                test_data = test_data.cuda()
                result = pinn(test_data).cpu()
                real_part = np.asarray(result[:, 0])
                imag_part = np.asarray(result[:, 1])
                ext_real, ext_imag = x.real, x.imag
                err = ((np.abs(real_part - ext_real)**2).sum()+ (np.abs(imag_part - ext_imag)**2).sum()) / len(x_axis)
                writer.add_scalar("L2 error %d" % N, err, epoch)
            pinn.train()    
        writer.add_scalar("train %d" % N, optimizer.step(closure).item(), epoch)
    print(err)
    err_all.append(err)
    writer.close()